In [16]:
import numpy as np
import pandas as pd

In [17]:
train_ref = 'train.csv'
test_ref = 'test.csv'

In [18]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean', weighted=False):
        self.k = k
        self.distance_metric = distance_metric
        self.weighted = weighted

    def fit(self, X, y):
        # TODO: Implement the fit method
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        # TODO: Implement the predict method
        predictions = []
        for x in X:
            distances = self.compute_distance(self.X_train, x)
            nn_indices = np.argsort(distances)[:self.k]
            nn_labels = self.y_train[nn_indices]
            nn_distances = distances[nn_indices]

            if self.weighted:
                inverse_distances = 1 / (nn_distances + 1e-5) 
                weighted_votes = np.zeros(np.max(self.y_train) + 1)
                for idx, label in enumerate(nn_labels):
                    weighted_votes[label] += inverse_distances[idx]
                prediction = np.argmax(weighted_votes)
            else:
                prediction = np.argmax(np.bincount(nn_labels.astype(int)))

            predictions.append(prediction)
        return np.array(predictions)

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation
        if self.distance_metric == 'euclidean':
            distances = np.sqrt(np.sum((X1 - X2) ** 2, axis=1))
            #Using euclidean distance formula
        return distances

In [19]:
# Define data preprocessing function
def preprocess_data(train_ref, test_ref):
    train_data = pd.read_csv(train_ref)
    test_data = pd.read_csv(test_ref)

    # TODO: Implement data preprocessing
    X_train = train_data.drop(columns=['Exited', 'id'])
    y_train = train_data['Exited'].astype(int) 

    X_test = test_data.drop(columns=['id'])

    X_train = X_train.select_dtypes(include=[np.number]).values
    X_test = X_test.select_dtypes(include=[np.number]).values
    y_train = y_train.values

    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

    return X_train, y_train, X_test

In [20]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    # TODO: Implement cross-validation
    n_samples = len(X)
    fold_size = n_samples // n_splits
    scores = []

    for i in range(n_splits):
        start_val = i * fold_size
        end_val = (i + 1) * fold_size if i < n_splits - 1 else n_samples

        X_val = X[start_val:end_val]
        y_val = y[start_val:end_val]

        X_train = np.concatenate((X[:start_val], X[end_val:]), axis=0)
        y_train = np.concatenate((y[:start_val], y[end_val:]), axis=0)

        knn.fit(X_train, y_train)

        y_pred = knn.predict(X_val)

        curScore = roc(y_val, y_pred)
        scores.append(curScore)

    return scores

def roc(y_true, y_pred):
    sorted_indices = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[sorted_indices]

    true_positives = []
    false_positives = []

    P = np.sum(y_true == 1) 
    N = np.sum(y_true == 0) 

    trueCount = 0
    falseCount = 0

    prev_label = -1
    for i in range(len(y_pred)):
        if y_pred[i] != prev_label:
            true_positives.append(trueCount / P)
            false_positives.append(falseCount / N)
            prev_label = y_pred[i]

        if y_true_sorted[i] == 1:
            trueCount += 1
        else:
            falseCount += 1

    true_positives.append(1)
    false_positives.append(1)

    tpArr = np.array(true_positives)
    fpArr = np.array(false_positives)
    auc = np.trapz(tpArr, fpArr)

    return auc

In [ ]:
X, y, X_test = preprocess_data(train_ref, test_ref)

knn = KNN(k=5, distance_metric='euclidean')

cv_scores = cross_validate(X, y, knn)

print("Cross-validation scores:", cv_scores)

ks = [3, 5, 7, 9, 11, 13, 15, 17, 19]
high_score = -1
high_k = None

for k in ks:
    knn = KNN(k=k, distance_metric='euclidean', weighted=True)
    cv_scores = cross_validate(X, y, knn)
    avg_score = np.mean(cv_scores) 

    if avg_score > high_score:
        high_score = avg_score
        high_k = k

print(f"Optimal K: {high_k}, of score: {high_score}")
# TODO: Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(k=high_k, distance_metric='euclidean', weighted=True)
knn.fit(X, y)
test_predictions = knn.predict(X_test)

# Save test predictions
pd.DataFrame({'id': pd.read_csv(test_ref)['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)

CV Scores: [0.7580419789941601, 0.7635930270797424, 0.7651226520439236, 0.7620617005163131, 0.7435775682165994]
